<a href="https://colab.research.google.com/github/Anphamthanh/LLMs-from-scratch/blob/main/Step1_synData_FAQ_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet -U langchain chromadb langchain-openai pypdf gradio datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
loader = PyPDFLoader("/content/How_to_access_CINECA_HPC_facility.pdf") #điền path đến file của bạn
splits = loader.load_and_split(text_splitter)

In [ ]:
synthetic_prompt = """
Bạn sẽ được cung cấp với một đoạn văn bản. Nhiệm vụ của bạn là hãy dựa trên nội dung này và tạo ra 20 cặp câu hỏi theo dạng FAQ mà một người bình thường sẽ hỏi:
---CONTEXT---
{context}
---END---
Bạn hãy tạo ra 20 cặp câu hỏi theo dạng FAQ mà một người bình thường sẽ hỏi với format như sau
---FORMAT INSTRUCTION---
Question:
<question1>
Answer:
<answer1>

Question:
<question2>
Answer:
<answer2>
... do this 10 times
---END---
Now, let's start
---START---
"""

synthetic_prompt_vi = """
Bạn sẽ được cung cấp với một đoạn văn bản. Nhiệm vụ của bạn là hãy dựa trên nội dung này và tạo ra 20 cặp câu hỏi theo dạng FAQ mà một người bình thường sẽ hỏi:
---CONTEXT---
{context}
---END---
Bạn hãy tạo ra 20 cặp câu hỏi theo dạng FAQ mà một người bình thường sẽ hỏi với format như sau
---FORMAT INSTRUCTION---
Question:
<question1>
Answer:
<answer1>

Question:
<question2>
Answer:
<answer2>
... do this 10 times
---END---
Now, let's start. Hãy nhớ rằng các cặp Question và Answer được tạo ra bằng tiếng Việt.
---START---
"""

In [ ]:
splits[0].page_content

'How to access CINECA HPC facility.  \n \nGeneral information concerning access to CINECA facilities can be found here:  \nhttps://wiki.u -gov.it/confluence/display/SCAIUS/UG2.1+Getting+started  \n \nParticularly, the steps to follow to get access to our HPC system  LEONARDO  are: \n1. You have to first register in our UserDB (unless already registered) https://userdb.hpc.cineca.it/ , \nclick on "Create new user" and fill in the form with your p ersonal Data.'

In [ ]:
prompt_list = []

# Tạo prompt để generate synthetic data với toàn file PDF

for split in splits:
  gen_prompt = synthetic_prompt.format(context=split.page_content)
  # gen_prompt = synthetic_prompt_vi.format(context=split.page_content) # dùng dòng này nếu muốn QA bằng tiếng Việt
  prompt_list.append(gen_prompt)

In [ ]:
print(gen_prompt)


Bạn sẽ được cung cấp với một đoạn văn bản. Nhiệm vụ của bạn là hãy dựa trên nội dung này và tạo ra 20 cặp câu hỏi theo dạng FAQ mà một người bình thường sẽ hỏi:
---CONTEXT---
d. Move to the tab "edit";  
e. Once there, move to the section "collaborators" and add each collaborator by typing his 
surname and selecting it from the menu.  
f. Click on “save” button.  
Please note that, collaborators are not added "by default", i.e. if they are simply registered on the  
userdb.  
It is your responsibility, as PI, to add them to the project.  
Please contact superc@cineca.it  in case you need assistance  to access  to the cluster.
---END---
Bạn hãy tạo ra 20 cặp câu hỏi theo dạng FAQ mà một người bình thường sẽ hỏi với format như sau
---FORMAT INSTRUCTION---
Question:
<question1>
Answer:
<answer1>

Question:
<question2>
Answer:
<answer2>
... do this 10 times
---END---
Now, let's start
---START---



In [ ]:
from openai import OpenAI
from google.colab import userdata
userdata.get('OPENAI_API_KEY')
llm = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
list_qa = []
# thời gian chờ sẽ khá lâu, trang pdf càng dài thì thời gian chờ càng lâu
for prompt in prompt_list:
  response = llm.chat.completions.create(model='gpt-3.5-turbo',
              messages=[{"role": "user", "content": prompt}],
              temperature=0.1,)
  list_qa.append(response.choices[0].message.content)

In [ ]:
# thử cặp qa đầu tiên

print(list_qa[0])

Question:
How can I access the CINECA HPC facility?
Answer:
You can access the CINECA HPC facility by following the steps outlined in the provided link: https://wiki.u-gov.it/confluence/display/SCAIUS/UG2.1+Getting+started

Question:
Where can I find general information about accessing CINECA facilities?
Answer:
You can find general information about accessing CINECA facilities on the provided link: https://wiki.u-gov.it/confluence/display/SCAIUS/UG2.1+Getting+started

Question:
Do I need to register in the UserDB to access the HPC system LEONARDO?
Answer:
Yes, you need to register in the UserDB to access the HPC system LEONARDO. You can do so by clicking on "Create new user" on the following link: https://userdb.hpc.cineca.it/

Question:
What information do I need to provide when registering in the UserDB?
Answer:
When registering in the UserDB, you need to fill in the form with your personal data.

Question:
Can I access the HPC system LEONARDO without registering in the UserDB?
Answ

In [ ]:
def parse_qa_pairs(text):
    """
    Parses a given text containing questions and answers into a list of dictionaries.

    Parameters:
    - text (str): A string containing questions and answers in a structured format.

    Returns:
    - List[Dict[str, str]]: A list of dictionaries, each representing a question-answer pair.
    """
    qa_pairs_simplified = []

    # Split the text based on "Question:" as a delimiter and ignore the first split which is empty
    sections = text.split("Question:\n")[1:]

    for section in sections:
        # Each section contains one question and one answer split by "Answer:"
        question_part, answer_part = section.split("\nAnswer:\n")
        question = question_part.strip()
        answer = answer_part.strip()
        qa_pairs_simplified.append({"question": question, "answer": answer})

    return qa_pairs_simplified

In [ ]:
qa_pairs = []

for qa in list_qa:
  pairs = parse_qa_pairs(qa)
  for pair in pairs:
    qa_pairs.append(pair)

In [ ]:
qa_pairs[0]

{'question': 'How can I access the CINECA HPC facility?',
 'answer': 'You can access the CINECA HPC facility by following the steps outlined in the provided link: https://wiki.u-gov.it/confluence/display/SCAIUS/UG2.1+Getting+started'}

In [ ]:
len(qa_pairs)

70

##Convert to ShareGPT

In [ ]:
sharegpt_data = []
for pair in qa_pairs:
  convo = {}
  convo['conversations'] = [{"from": "human", "value": pair['question']}, {"from": "gpt", "value": pair['answer']}]
  sharegpt_data.append(convo)

In [ ]:
sharegpt_data

[{'conversations': [{'from': 'human',
    'value': 'How can I access the CINECA HPC facility?'},
   {'from': 'gpt',
    'value': 'You can access the CINECA HPC facility by following the steps outlined in the provided link: https://wiki.u-gov.it/confluence/display/SCAIUS/UG2.1+Getting+started'}]},
 {'conversations': [{'from': 'human',
    'value': 'Where can I find general information about accessing CINECA facilities?'},
   {'from': 'gpt',
    'value': 'You can find general information about accessing CINECA facilities on the provided link: https://wiki.u-gov.it/confluence/display/SCAIUS/UG2.1+Getting+started'}]},
 {'conversations': [{'from': 'human',
    'value': 'Do I need to register in the UserDB to access the HPC system LEONARDO?'},
   {'from': 'gpt',
    'value': 'Yes, you need to register in the UserDB to access the HPC system LEONARDO. You can do so by clicking on "Create new user" on the following link: https://userdb.hpc.cineca.it/'}]},
 {'conversations': [{'from': 'human',
 

In [ ]:
# prompt: save sharegpt_data to a json file with enforce anscii = false

import json

with open('sharegpt_data.json', 'w', encoding='utf-8') as f:
  json.dump(sharegpt_data, f, indent=4, ensure_ascii=False)

In [ ]:
import datasets

dataset = datasets.load_dataset('json', data_files='sharegpt_data.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset.push_to_hub('huggingface_username/dataname') # điền tên huggingface và dataset_name, nhớ set HF_TOKEN trong environment

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/qnguyen3/demo_faq/commit/e4ebda7555b8e3f5f1232dd2d05eea52f92cf431', commit_message='Upload dataset', commit_description='', oid='e4ebda7555b8e3f5f1232dd2d05eea52f92cf431', pr_url=None, pr_revision=None, pr_num=None)